In [8]:
from openai import OpenAI
import asyncio
from ollama import AsyncClient
import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
#test ollama

async def chat():
    """
    Stream a chat from Llama using the AsyncClient.
    """
    message = {
        "role": "user",
        "content": "Tell me an interesting fact about elephants"
    }
    async for part in await AsyncClient().chat(
        model="llama3", messages=[message], stream=True
    ):
        print(part["message"]["content"], end="", flush=True)


asyncio.run(chat())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [4]:
def get_chatbot_response(client, mode_name, messages, temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000
    ).choices[0].message.content

    return response

In [ ]:
client = OpenAI(
    api_key=os.environ.get("RUN_POD_TOKEN"),
    base_url=os.getenv("RUN_POD_BASE_URL")
)
model_name= os.getenv("RUN_POD_MODEL_NAME")


In [ ]:
messages = [{"role":"system", "content": "What is the capital of Germany"}]
response = get_chatbot_response(client, model_name, messages)
print(response)

## Prompt Engineering 
### Structured Output

In [ ]:
system_prompt = """
 You are a helpful assistant that answers questions about capitals of countries

 Your output should be in a structured format like the one below. You are not allowed to write anything other than the json object:

[{
    "country": the country that you will give the capital of,
    "capital": the capital of the country stated}]
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": "What is the capital of Spain"})
response = get_chatbot_response(client, model_name, messages)
print(response)


In [ ]:
## convert the response to a json object
json_response = json.loads(response)
json_response

## Input Structuring

In [ ]:
user_inout = """
Get me the capital fo the following countries
```
1. Kenya
2. Ukraine
3. Spain
4. Poland
4. Germany
```
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": user_inout})
response = get_chatbot_response(client, model_name, messages)

## convert the response to a json object
json_response = json.loads(response)
json_response


### Give the model time to think - (Chain of thought)

In [ ]:
user_prompt = """
Calculate the result of this eqaution: 1 + 3

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
 result: The final number resulted from the calculation of the equation above
}
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

### Decrease the margin of error


In [ ]:
user_prompt = """
Calculate the result of this eqaution: 259 + 341 - 100 * 20008 / 0.5 + 27683 / 2 * 365

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
 steps: This is where you solve the equation bit by bit following the BEDMAS rule of order of operations. You need to show your work and calculate each step leading into the final results on feel free to write in free text
 result: The final number resulted from the calculation of the equation above
}
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

### RAG - (Retrival Augmented Generation) model

In [ ]:
user_prompt = """

What is new in iphone 16?

"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

In [ ]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

user_prompt = """  
{iphone_16}

What is new in iphone 16?
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)